In [ ]:
import githubwebstats, query_ads
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
github_urls = githubwebstats._clean_github_urls()

In [ ]:
results = pd.DataFrame(columns=['github_url', 'repo_name', 'metions', 'n_unq_authors', 'n_unq_first_authors',
                                'arxiv_ids', 'citation_count', 'unq_keywords'])

for idx in tqdm(range(len(github_urls))):
    github_url = github_urls[idx]
    repo_name = github_url.split('/')[-1]
    if len(repo_name) < 3:
        continue

    r = query_ads.query_ads("full:' " + repo_name +" '")

    results.loc[idx, 'github_url'] = github_url
    results.loc[idx, 'repo_name'] = repo_name
    results.loc[idx, 'metions'] = len(r)
    results.loc[idx, 'n_unq_authors'] = len(np.unique([item for sublist in [r1['author'] for r1 in r if r1['author'] is not None] for item in sublist]))
    results.loc[idx, 'n_unq_first_authors'] = len(np.unique([r1['first_author'] for r1 in r if r1['first_author'] is not None]))


    results.loc[idx, 'unq_keywords'] = np.unique([item for sublist in [r1['keyword'] for r1 in r if r1['keyword'] is not None] for item in sublist])
    results.loc[idx, 'citation_count'] = np.sum([r1['citation_count'] for r1 in r])
    results.loc[idx, 'arxiv_ids'] = [r1['arxiv_id'] for r1 in r if r1['arxiv_id'] is not None]

In [ ]:
results.to_csv('repo_success_metrics.csv')

In [ ]:
results